In [28]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"
import sys
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer
import json
import torch
import re
from tqdm import tqdm
from collections import Counter

In [29]:
import openai

In [30]:
openai.api_key_path = '/data4/polyakov/GPT_KEY_MY_2'

In [31]:
# base_model = '/data4/polyakov/instruction_tuning/data/Llama-2-7b-chat-hf'
# model = AutoModelForCausalLM.from_pretrained(
#         base_model,
#         load_in_4bit=True,
#         local_files_only=True,
#         device_map='auto',
# )
# model.config.use_cache = False
# tokenizer = AutoTokenizer.from_pretrained(base_model, local_files_only=True)

In [32]:
def make_gpt3_request(prompt: str):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
                {"role": "system", "content": "You are a helpful assistant, useful for error detection."},
                {"role": "user", "content": prompt},
            ],
        stop=None,
        )
    return response

In [33]:
def make_gpt4_request(prompt: str):
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[
                {"role": "system", "content": "You are a helpful assistant, useful for error detection."},
                {"role": "user", "content": prompt},
            ],
        stop=None,
        )
    return response

In [15]:
make_gpt3_request('Hi!')['choices'][0]['message']['content']

RateLimitError: You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.

In [ ]:
make_gpt4_request('Hi!')['choices'][0]['message']['content']

'Hello! How can I assist you today?'

In [16]:
# curr_prompt = "What does the fllowing sentence say? Translate to english. \u6c7d\u6c34\u74f6\u5851\u6599\u676f\u900f\u660e\u78e8\u7802\u4fbf\u643a\u4e0d\u6613\u6454\u7834\u5bc6\u5c01\u540a\u5e26\u5976\u8336\u676f\u6c34\u676f\u505a\u793c\u54c1 Translation:"
# tokenized = tokenizer(curr_prompt, return_tensors="pt").input_ids
# res = model.generate(tokenized, max_length=tokenized.shape[1] + 2)
# print(tokenizer.batch_decode(res)[0].removeprefix(tokenizer.batch_decode(tokenized)[0]))

In [17]:
prompt = '''
You are you will be given api calls from some model and api outputs from rapidapi apis. 
Sometimes api return errors. 
It is hard to parse them using rule-based methods. 
An api may respond with an empty output (it could be empty list, or an empty dict), contain a field that states it is an error or etc.
You will be given 5 examples. You should respond only "Yes" if the response does not contain errors from an api and "No" if not.
Please make shure that api returned what the api call requested. If not, return "No"
Examples:
API Call:
Thought: To fulfill the user's request for gathering user information and recent media posts from Instagram for a specific username 'nike', I previously called the 'userinfo_for_instagram_cheapest' function with the argument {"username": "nike"}. By doing so, I efficiently retrieved the desired user data, including their biography, bio links, biographical information, and other relevant details. Now, to proceed with accessing the user's recent media posts, I will call the 'user_medias_for_instagram_cheapest' function with the argument {"user_id": "113294420064920"}. This will allow me to retrieve the most recent media posts of the user 'nike' and provide the user with the requested data effectively.
Action: user_medias_for_instagram_cheapest
Action Input: {
  "user_id": "113294420064920"
}
API Response:
{"error": "", "response": "{'items': [], 'num_results': 0, 'status': 'ok'}"}
Result:
No
###
API Call:
Thought: The user requested the list of languages supported by the LanguageTool API for grammar and writing assistance. To fulfill this request, I need to call the "list_of_languages_for_languagetool" function without any arguments. By doing so, I expect to receive the complete list of supported languages. This information will be crucial for implementing multilingual support in the project, ensuring accurate grammar and writing assistance for different languages.
Action: list_of_languages_for_languagetool
Action Input: {}
API Response:
{"error": "", "response": "[{'name': 'Arabic', 'code': 'ar', 'longCode': 'ar'}, {'name': 'Asturian', 'code': 'ast', 'longCode': 'ast-ES'}, {'name': 'Belarusian', 'code': 'be', 'longCode': 'be-BY'}, {'name': 'Breton', 'code': 'br', 'longCode': 'br-FR'}, {'name': 'Catalan', 'code': 'ca', 'longCode': 'ca-ES'}, {'name': 'Catalan (Valencian)', 'code': 'ca', 'longCode': 'ca-ES-valencia'}, {'name': 'Catalan (Balearic)', 'code': 'ca', 'longCode': 'ca-ES-balear'}, {'name': 'Danish', 'code': 'da', 'longCode': 'da-DK'}, {'name': 'German', 'code': 'de', 'longCode': 'de'}, {'name': 'German (Germany)', 'code': 'de', 'longCode': 'de-DE'}, {'name': 'German (Austria)', 'code': 'de', 'longCode': 'de-AT'}, {'name': 'German (Swiss)', 'code': 'de', 'longCode': 'de-CH'}, {'name': 'Simple German', 'code': 'de-DE-x-simple-language', 'longCode': 'de-DE-x-simple-language'}, {'name': 'Greek', 'code': 'el', 'longCode': 'el-GR'}, {'name': 'English', 'code': 'en', 'longCode': 'en'}, {'name': 'English (US)', 'code': 'en', 'longCode': 'en-U..
Result:
Yes
###
API Call:
Thought: I will now call the "dividends_for_freedom_finance" function to retrieve information about dividends for the symbol "AMZN". By using the argument {"symbol": "AMZN"}, I can focus specifically on the dividends related to the stock "AMZN". This function will help me analyze the dividend history for "AMZN" and provide the user with the requested information.
Action: dividends_for_freedom_finance
Action Input: {
"symbol": "AMZN"
}
API Response:
{"error": "", "response": "{'total': 0, 'offset': 0, 'results': [], 'responseStatus': None}"}
Result:
No
###
API Call:
Thought: To gather the required information for integrating APIs into the company's website, I can call the "list_all_api_for_testing_for_my_use" function. This function will provide me with descriptions, methods, required parameters, and optional parameters for each API. With this information, I can analyze and compare the APIs to make an informed decision.
Action: list_all_api_for_testing_for_my_use
Action Input: {}
API Response:
{"error": "Unauthorized error...", "response": "{'1forge.com': {'added': '2017-05-30T08:34:14.000Z', 'preferred': '0.0.1', 'versions': {'0.0.1': {'added': '2017-05-30T08:34:14.000Z', 'info': {'contact': {'email': 'contact@1forge.com', 'name': '1Forge', 'url': 'http://1forge.com'}, 'description': 'Stock and Forex Data and Realtime Quotes', 'title': '1Forge Finance APIs', 'version': '0.0.1', 'x-apisguru-categories': ['financial'], 'x-logo': {'backgroundColor': '#24292e', 'url': 'https://api.apis.guru/v2/cache/logo/https_1forge.com_assets_images_f-blue.svg'}, 'x-origin': [{'format': 'swagger', 'url': 'http://1forge.com/openapi.json', 'version': '2.0'}], 'x-providerName': '1forge.com'}, 'updated': '2017-06-27T16:49:57.000Z', 'swaggerUrl': 'https://api.apis.guru/v2/specs/1forge.com/0.0.1/swagger.json', 'swaggerYamlUrl': 'https://api.apis.guru/v2/specs/1forge.com/0.0.1/swagger.yaml', 'openapiVer': '2.0', 'link': 'https://api.apis.guru/v2/specs/1forge.com/0.0.1.json'}}}, '1password.com:events': {'added': '2021-07-19...
Result:
No
###
API Call:
Thought: I need to gather user information for a specific Instagram username, 'nike', as requested by the user. They also want access to the user's recent media posts. To achieve this, I will call the 'userinfo_for_instagram_cheapest' function with the argument of {"username": "nike"}. This will help me retrieve the desired data efficiently and fulfill the user's request effectively.
Action: userinfo_for_instagram_cheapest
Action Input: {
  "username": "nike"
}
API Response:
{"error": "", "response": "{'data': {'user': {'biography': 'Spotlighting athlete* and\\xa0\ud83d\udc5f\\xa0stories\\n#BlackLivesMatter, #StopAsianHate, and #NoPrideNoSport', 'bio_links': [{'title': '', 'lynx_url': 'https://l.instagram.com/?u=https%3A%2F%2Flinkin.bio%2Fnike&e=AT063fG4N158ZbZOaZ_A32batiXwR-0Yih5XNa5wfSNPCiDo546PSV0a8mm72ElVxWgNJdgYeHNOTMMTOe_kn7R-C-f2N9ym', 'url': 'https://linkin.bio/nike', 'link_type': 'external'}], 'biography_with_entities': {'raw_text': 'Spotlighting athlete* and\\xa0\ud83d\udc5f\\xa0stories\\n#BlackLivesMatter, #StopAsianHate, and #NoPrideNoSport', 'entities': [{'user': None, 'hashtag': {'name': 'nopridenosport'}}, {'user': None, 'hashtag': {'name': 'stopasianhate'}}, {'user': None, 'hashtag': {'name': 'blacklivesmatter'}}]}, 'blocked_by_viewer': False, 'restricted_by_viewer': None, 'country_block': False, 'eimu_id': '113294420064920', 'external_url': 'https://linkin.bio/nike', 'external_url_linkshimmed': 'https://l.instagram.com/?u=https%3A%2F%2Flinkin.bio%2Fnike&e=AT04S69...
Result:
Yes
###
API Call: 
<api_call>
API Response:
<api_response>
Result:
'''

In [18]:
prompt_zero_shot = '''
You are you will be api calls from some model and api outputs from rapidapi apis. 
Sometimes api return errors. 
It is hard to parse them using rule-based methods. 
An api may respond with an empty output (it could be empty list, or an empty dict), contain a field that states it is an error or etc.
You will be given 5 examples. You should respond only "Yes" if the response does not contain errors from an api and "No" if not.
Please make shure that api returned what the api call requested. If not, return "No"
You will be given data in the folllowing format.
API Call: 
api_call
API Response:
api_response
Result: 
Yes/No
Begin!
###
API Call: 
<api_call>
API Response:
<api_response>
Result:
'''

In [5]:
data = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/cleaned_toolllama_G123_dfs_train_downloaded1311_no_this_is_not_the_first_time_you_tried.json', 'r'))

In [37]:
for item in data[:10]:
    flag = True
    for i, step in enumerate(item['conversations']):
        if step['from'] == 'function':
            curr_prompt = prompt.replace('<api_call>', item['conversations'][i-1]['value'])
            curr_prompt = curr_prompt.replace('<api_response>', step['value'])
            with torch.no_grad():
                tokenized = tokenizer(curr_prompt, return_tensors="pt").input_ids
                res = model.generate(tokenized, max_length=tokenized.shape[1] + 2)
                print('API Call:')
                print(item['conversations'][i-1]['value'])
                print('API Response:')
                print(step['value'])
                print('Result:')
                print(tokenizer.batch_decode(res)[0].removeprefix(tokenizer.batch_decode(tokenized)[0]))
                print('#' * 100)

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


API Call:

Thought: I need to gather user information for a specific Instagram username, 'nike', as requested by the user. They also want access to the user's recent media posts. To achieve this, I will call the 'userinfo_for_instagram_cheapest' function with the argument of {"username": "nike"}. This will help me retrieve the desired data efficiently and fulfill the user's request effectively.
Action: userinfo_for_instagram_cheapest
Action Input: {
  "username": "nike"
}
API Response:
{"error": "", "response": "{'data': {'user': {'biography': 'Spotlighting athlete* and\\xa0\ud83d\udc5f\\xa0stories\\n#BlackLivesMatter, #StopAsianHate, and #NoPrideNoSport', 'bio_links': [{'title': '', 'lynx_url': 'https://l.instagram.com/?u=https%3A%2F%2Flinkin.bio%2Fnike&e=AT063fG4N158ZbZOaZ_A32batiXwR-0Yih5XNa5wfSNPCiDo546PSV0a8mm72ElVxWgNJdgYeHNOTMMTOe_kn7R-C-f2N9ym', 'url': 'https://linkin.bio/nike', 'link_type': 'external'}], 'biography_with_entities': {'raw_text': 'Spotlighting athlete* and\\xa0\u

KeyboardInterrupt: 

Прогоню модель на тесте, который у меня получился

In [19]:
with open('/data4/polyakov/instruction_tuning/ToolBench_pure_1/data/self_correction/error_test_data/test_marked.txt', 'r') as f:
    test_data = f.read()
test_items = test_data.split('\n###################################################################\n')[:-1]

In [20]:
# predictions = []
# for item in tqdm(test_items):
#     api_call = item.split('API Call: \n')[1].split('\nAPI Response')[0]
#     api_response = item.split('API Response:\n')[1].split('\nResult:')[0]
#     curr_prompt = prompt.replace('<api_call>', api_call)
#     curr_prompt = curr_prompt.replace('<api_response>', api_response)
#     with torch.no_grad():
#         tokenized = tokenizer(curr_prompt, return_tensors="pt").input_ids
#         res = model.generate(tokenized, max_length=tokenized.shape[1] + 1)
#         pred = tokenizer.batch_decode(res)[0].removeprefix(tokenizer.batch_decode(tokenized)[0])
#         predictions.append(pred)

In [21]:
predictions = []
for item in tqdm(test_items):
    api_call = item.split('API Call: \n')[1].split('\nAPI Response')[0]
    api_response = item.split('API Response:\n')[1].split('\nResult:')[0]
    curr_prompt = prompt.replace('<api_call>', api_call)
    curr_prompt = curr_prompt.replace('<api_response>', api_response)
    with torch.no_grad():
        tokenized = tokenizer(curr_prompt, return_tensors="pt").input_ids
        res = model.generate(tokenized, max_length=tokenized.shape[1] + 1)
        pred = tokenizer.batch_decode(res)[0].removeprefix(tokenizer.batch_decode(tokenized)[0])
        predictions.append(pred)

  0%|                                                                                              | 0/100 [00:00<?, ?it/s]


NameError: name 'tokenizer' is not defined

In [26]:
Counter(predictions)

Counter({'Yes': 77, 'No': 23})

In [20]:
answers = []
for item in test_items:
    ans = item.split('Result:\n')[1].strip()
    answers.append(ans)

In [24]:
acc = 0
fp = 0
fn = 0
tp = 0
tn = 0
for gt, pred in zip(answers, predictions):
    if gt == pred:
        acc += 1
        if gt == 'No':
            tn += 1
        else:
            tp += 1
    elif gt == 'No' and pred == 'Yes':
        fp += 1
    else:
        fn += 1
print(f'Accuracy: {acc}, Precision: {tp / (tp + fp)}, Recall: {tp / (tp + fn)}')

Accuracy: 88, Precision: 0.974025974025974, Recall: 0.8823529411764706


Теперь посмотрим на результаты на новом тесте

In [22]:
with open('/data4/polyakov/instruction_tuning/ToolBench_pure_1/data/self_correction/error_test_data/final_test_balanced_marked.json', 'r') as f:
    balanced_test = json.load(f)

balanced_test[4]

{'query': "Step 4: I'm looking for the external information of a device-asset identified by '71491234-A'. Could you assist me in retrieving the code, status, and message associated with this identifier? Additionally, please let me know if there are any optional parameters available.\n",
 'api_call': 'Thought: I think the current API call is made to retrieve the information of a device-asset with the identifier "71491234-A". The reason for this call is to fulfill the user\'s request to fetch the code, status, and message associated with the given identifier. The arguments for the API call specify the "identifier" parameter as "71491234-A". By providing this identifier, we can retrieve the desired device-asset information from the Vikkon platform.\nAction: get_device_asset_by_identifier_copy_for_basic_info_vikkon_assets\nAction Input: {\n  "identifier": "71491234-A"\n}\n',
 'api_response': '{"error": "", "response": "{\'code\': \'404 not found\', \'status\': 404, \'message\': \'Resource 

In [23]:
# predictions_balanced_test = []
# for item in tqdm(balanced_test):
#     api_call = item['api_call']
#     api_response = item['api_response']
#     curr_prompt = prompt_zero_shot.replace('<api_call>', api_call)
#     curr_prompt = curr_prompt.replace('<api_response>', api_response)
#     with torch.no_grad():
#         tokenized = tokenizer(curr_prompt, return_tensors="pt").input_ids
#         res = model.generate(tokenized, max_length=tokenized.shape[1] + 1)
#         pred = tokenizer.batch_decode(res)[0].removeprefix(tokenizer.batch_decode(tokenized)[0])
#         predictions_balanced_test.append(pred)

In [24]:
import time

In [27]:
predictions_balanced_test = []
for item in tqdm(balanced_test):
    api_call = item['api_call']
    api_response = item['api_response']
    # curr_prompt = prompt_zero_shot.replace('<api_call>', api_call)
    curr_prompt = prompt.replace('<api_call>', api_call)
    curr_prompt = curr_prompt.replace('<api_response>', api_response)
    # pred = make_gpt3_request(curr_prompt)['choices'][0]['message']['content']
    # pred = make_gpt4_request(curr_prompt)['choices'][0]['message']['content']
    # predictions_balanced_test.append(pred)
    # time.sleep(2)

    break
    # with torch.no_grad():
    #     tokenized = tokenizer(curr_prompt, return_tensors="pt").input_ids
    #     res = model.generate(tokenized, max_length=tokenized.shape[1] + 1)
    #     pred = tokenizer.batch_decode(res)[0].removeprefix(tokenizer.batch_decode(tokenized)[0])
    #     predictions_balanced_test.append(pred)

  0%|                                                                                              | 0/100 [00:00<?, ?it/s]
ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/data2/polyakov/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/jupyter_client/session.py", line 95, in json_packer
    return json.dumps(
UnicodeEncodeError: 'utf-8' codec can't encode characters in position 5851-5852: surrogates not allowed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data2/polyakov/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/zmq/eventloop/zmqstream.py", line 584, in _run_callback
    f = callback(*args, **kwargs)
  File "/data2/polyakov/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/ipykernel/iostream.py", line 120, in _handle_event
    event_f()
  File "/data2/polyakov/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/ipykernel/iostream.

In [ ]:
curr_prompt

In [44]:
predictions_balanced_test

['No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes_contender',
 'Yes',
 'No',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'Yes',
 'No',
 'No',
 'No',
 'Yes',
 'Yes',
 'No',
 'Yes',
 'No']

In [8]:
len(predictions_balanced_test)

NameError: name 'predictions_balanced_test' is not defined

In [45]:
answers_balanced_test = [item['label'] for item in balanced_test]

acc = 0
fp = 0
fn = 0
tp = 0
tn = 0
for gt, pred in zip(answers_balanced_test, predictions_balanced_test):
    if gt == pred:
        acc += 1
        if gt == 'Yes':
            tn += 1
        else:
            tp += 1
    elif gt == 'Yes' and pred == 'No':
        fp += 1
    else:
        fn += 1
print(f'Accuracy: {acc}, Precision: {tp / (tp + fp)}, Recall: {tp / (tp + fn)}')

Accuracy: 75, Precision: 0.6226415094339622, Recall: 0.868421052631579
